In [ ]:
import os
import sys
import csv
import time
import chess
import chess.engine
import requests
import json
from dotenv import load_dotenv
from subprocess import Popen, PIPE, DEVNULL
from tqdm import tqdm

In [ ]:
# Constants
stockfish_exe = "/Users/saumik/Documents/classes/rotations/code/Stockfish/src/stockfish"
engine = chess.engine.SimpleEngine.popen_uci(stockfish_exe)
depth = 20
load_dotenv()   
token = os.getenv('LICHESS_TOKEN')

In [ ]:
def get_fens(player, max_games='null', token=None):
    headers = {'Accept':'application/x-ndjson'}
    if token:
        headers['Authorization'] = f'Bearer {token}'
    params = {'max':max_games, 'perfType':'blitz'}
    response = requests.get(f"https://lichess.org/api/games/user/{player}", headers=headers, params=params)
    with open(f'data/{player}.csv', 'w') as f:
        for line in response.iter_lines():
            line = json.loads(line.decode('utf-8'))
            if line['variant'] == 'standard' and 'moves' in line:
                try:
                    color = 'w' if line['players']['white']['user']['name'] == player else 'b'
                except:
                    continue
                moves = line['moves'].split()
                board = chess.Board()
                for san in moves:
                    uci = board.parse_san(san).uci()
                    fen = board.fen()
                    movenum = int(fen.split()[5])
                    if fen.split()[1] == color and movenum >= 10 and movenum < 30:
                        print(f'{uci},{fen}', file=f)
                    board.push_uci(uci)

In [ ]:
players_to_download = ['Konevlad','Crecent','LongLive16Hayastan','may6enexttime','IWANNABEADOORED',
                       'Ogrilla','dolar9','MilfSalvatore','cutemouse83','Benefactorr',
                       'cjota95','alireza2003','vistagausta','GRUrussia','Drvitman',
                       'NIndja64','KnezMihailova','Bestinblitz','AnonLondonKiller','black_knight22',
                       'IVK88','Inventing_Invention','temp006','Arka50','Ernst_Gruenfeld',
                       'Zaola420','GeorgMeier','SindarovGM','BenjaminBokTwitch','Hungry_Dragon',
                       'avantage_ru','Chesstoday','GABUZYAN_CHESSMOOD','Tryhard00','TilChess',
                       'muisback','MikeGScarn','Zhalmakhanov_R','L04d1ng','BabaRamdev',
                       'opperwezen','Feokl1995','LucaBrazzi','Venost17','RebeccaHarris',
                       'gmmoranda','YuQuesada','FakeBruceLee','swimmerchess','GrigorGrigorov']

In [ ]:
# for p in tqdm(players_to_download):
#     get_fens(p, max_games=100, token=token)

In [ ]:
def clean_score(score):
    score = round(float(score),2)
    score = 2.0 if score > 2 else score
    score = -2.0 if score < -2 else score
    return score

def evaluate(fen, next_move=None):
    process = Popen(stockfish_exe, shell=True, stdin=PIPE, stdout=PIPE, stderr=DEVNULL)
    if next_move:
        output = process.communicate(input=str.encode(f"position fen {fen} moves {next_move}\neval\n"))[0]
    else:
        output = process.communicate(input=str.encode(f"position fen {fen}\neval\n"))[0]
    if process.returncode != 0:
        sys.stderr.write(f"Failed: {stockfish_exe}\n")
        sys.exit(1)

    # parse for parameter output
    components = ['Material','Imbalance','Mobility','King safety','Threats','Passed','Space','Winnable']
    scores = []
    for line in output.decode("utf-8").split("\n"):
        if any(c in line for c in components):
            mg_score = line.split('|')[3].split()[0]
            mg_score = clean_score(mg_score)
            scores.append(mg_score)
    return tuple(scores)

In [ ]:
# Unused at the moment
def get_next_states(fen):
    board = chess.Board(fen)
    moves = [move.uci() for move in board.legal_moves]
    states = set()
    for move in tqdm(moves):
        scores = evaluate(fen, move)
        if scores != ():
            states.add(scores)
    return states

In [ ]:
def get_best_move(fen):
#     print(fen)
    board = chess.Board(fen=fen)
    info = engine.analyse(board, chess.engine.Limit(depth=20))
    return info['pv'][0]

In [ ]:
def rm_paren(tup):
    if tup == ():
        return ',,,,,,,'
    return ','.join(str(i) for i in tup)

In [ ]:
def get_avg_bias(player):    
    num_lines = sum(1 for line in open(f'data/{player}.csv','r'))
    with open(f'data/{player}.csv', 'r') as f:
        with open(f'diffs/{player}.csv','w') as fw:
            csv_reader = csv.reader(f, delimiter=',')
#             count = 0
            for uci,fen in tqdm(csv_reader, total=num_lines):
#                 if count >= 10:
#                     break
                current_eval = evaluate(fen)
                played_eval = evaluate(fen, uci)
                best = get_best_move(fen)
                best_eval = played_eval if str(best) == str(uci) else evaluate(fen, best)
                print(f'{fen},{rm_paren(current_eval)},{uci},{rm_paren(played_eval)},{best},{rm_paren(best_eval)}', file=fw)
#                 count+=1

In [ ]:
# for p in players_to_download:
#     print(f'Starting {p}')
#     time.sleep(1)
#     get_avg_bias(p)
#     time.sleep(1)